In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import pycaret
from pycaret.classification import *
from pycaret.datasets import get_data

In [13]:
%pip install 'pycaret'

Note: you may need to restart the kernel to use updated packages.


In [14]:

%pip install 'pycaret[full]'

  Using cached Flask-2.2.3-py3-none-any.whl.metadata (3.9 kB)
  Using cached werkzeug-2.3.8-py3-none-any.whl.metadata (4.1 kB)
  Using cached boto3-1.34.55-py3-none-any.whl.metadata (6.6 kB)
  Using cached evidently-0.2.8-py3-none-any.whl.metadata (1.3 kB)
  Using cached explainerdashboard-0.4.5-py3-none-any.whl.metadata (3.8 kB)
  Using cached fairlearn-0.7.0-py3-none-any.whl.metadata (7.3 kB)
  Using cached fastapi-0.110.0-py3-none-any.whl.metadata (25 kB)
  Using cached fugue-0.8.7-py3-none-any.whl.metadata (17 kB)
  Using cached gradio-4.19.2-py3-none-any.whl.metadata (15 kB)
  Using cached hyperopt-0.2.7-py2.py3-none-any.whl.metadata (1.7 kB)
  Using cached interpret-0.5.1-py3-none-any.whl.metadata (1.1 kB)
  Using cached kmodes-0.12.2-py2.py3-none-any.whl.metadata (8.1 kB)
  Using cached m2cgen-0.10.0-py3-none-any.whl.metadata (9.9 kB)
  Using cached mlflow-1.30.1-py3-none-any.whl.metadata (11 kB)
  Using cached mlxtend-0.23.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached mot

In [5]:
data = pd.read_csv('train.csv')
data

,DATETIMEDATA,PM25,O3,WS,TEMP,RH,WD
0,2024-01-01 00:00:00,16.5,16.000000,1,27.0,79.0,115.0
1,2024-01-01 01:00:00,33.6,11.326513,1,27.0,82.0,97.0
2,2024-01-01 02:00:00,34.2,11.326513,1,26.0,84.0,96.0
3,2024-01-01 03:00:00,20.1,11.326513,1,26.0,82.0,108.0
4,2024-01-01 04:00:00,16.5,1.000000,1,27.0,79.0,106.0
...,...,...,...,...,...,...,...
1411,2024-02-28 19:00:00,23.6,11.326513,1,29.0,64.0,184.0
1412,2024-02-28 20:00:00,25.2,11.326513,1,29.0,64.0,183.0
1413,2024-02-28 21:00:00,23.2,1.000000,1,29.0,64.0,159.0
1414,2024-02-28 22:00:00,24.7,3.000000,1,28.0,65.0,134.0


In [6]:

from pycaret.regression import *
clf1 = setup(data, target='PM25', session_id = 123)

,Description,Value
0,Session id,123
1,Target,PM25
2,Target type,Regression
3,Original data shape,"(1416, 7)"
4,Transformed data shape,"(1416, 7)"
5,Transformed train set shape,"(991, 7)"
6,Transformed test set shape,"(425, 7)"
7,Numeric features,5
8,Categorical features,1
9,Preprocess,True


In [7]:
best = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
en,Elastic Net,4.1559,28.3896,5.3160,0.0149,0.2866,0.2629,0.0070
huber,Huber Regressor,4.1462,28.6228,5.3380,0.0052,0.2878,0.2616,0.0100
omp,Orthogonal Matching Pursuit,4.1890,28.6899,5.3440,0.0044,0.2883,0.2654,0.0070
lasso,Lasso Regression,4.1705,28.7630,5.3514,0.0015,0.2883,0.2639,0.0110
llar,Lasso Least Angle Regression,4.1705,28.7630,5.3514,0.0015,0.2883,0.2639,0.0070
ridge,Ridge Regression,4.1769,28.9287,5.3670,-0.0044,0.2890,0.2642,0.0080
et,Extra Trees Regressor,4.1771,28.9355,5.3676,-0.0046,0.2890,0.2642,0.0860
gbr,Gradient Boosting Regressor,4.1789,28.9363,5.3677,-0.0047,0.2891,0.2644,0.0430
lar,Least Angle Regression,4.1776,28.9397,5.3680,-0.0048,0.2891,0.2642,0.0110
lr,Linear Regression,4.1776,28.9397,5.3680,-0.0048,0.2891,0.2642,0.2620


In [8]:
t_en = create_model('en')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,4.3714,28.8857,5.3745,0.0032,0.3175,0.3115
1,3.9175,30.0022,5.4774,-0.0096,0.2711,0.2325
2,4.1430,26.6804,5.1653,0.0016,0.2799,0.2580
3,4.3085,29.0552,5.3903,0.0257,0.2831,0.2594
4,4.1398,27.9136,5.2833,0.0238,0.2985,0.2755
5,4.3656,31.6537,5.6262,0.0233,0.2927,0.2666
6,3.9306,24.2149,4.9209,0.0109,0.2706,0.2448
7,4.1158,32.0332,5.6598,0.0195,0.2855,0.2470
8,3.5920,20.2372,4.4986,0.0427,0.2451,0.2186


In [9]:
tune_rf = tune_model(best)
bagged = ensemble_model(tune_rf , n_estimators=20)


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,21:10:15
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,4.3351,28.5576,5.3439,0.0145,0.3153,0.3081
1,3.9189,30.0097,5.4781,-0.0099,0.2711,0.2323
2,4.1271,26.3426,5.1325,0.0142,0.2785,0.2571
3,4.2848,28.6154,5.3493,0.0404,0.2810,0.2578
4,4.1058,27.4461,5.2389,0.0402,0.2966,0.2739
5,4.3321,31.3012,5.5947,0.0342,0.2911,0.2647
6,3.9347,24.1967,4.9190,0.0117,0.2706,0.2450
7,4.0893,31.6891,5.6293,0.0301,0.2841,0.2457
8,3.5672,19.7230,4.4411,0.0670,0.2421,0.2169


Fitting 10 folds for each of 10 candidates, totalling 100 fits


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,4.3344,28.5631,5.3444,0.0143,0.3152,0.3078
1,3.9170,29.9757,5.4750,-0.0087,0.2708,0.2319
2,4.1257,26.3422,5.1325,0.0143,0.2784,0.2569
3,4.2833,28.6208,5.3498,0.0402,0.2809,0.2575
4,4.1050,27.4318,5.2375,0.0407,0.2963,0.2735
5,4.3282,31.2677,5.5917,0.0352,0.2908,0.2643
6,3.9360,24.2064,4.9200,0.0113,0.2707,0.2454
7,4.0952,31.7189,5.6320,0.0292,0.2844,0.2462
8,3.5670,19.7160,4.4403,0.0674,0.2420,0.2169


In [10]:
test = pd.read_csv('test.csv')
predict_model(bagged,data=test)

,DATETIMEDATA,O3,WS,TEMP,RH,WD,PM25,prediction_label
0,2024-02-29 00:00:00,3.0,1,28.0,66.0,91.0,18.500000,18.628939
1,2024-02-29 01:00:00,24.0,1,27.0,68.0,96.0,16.100000,17.953123
2,2024-02-29 02:00:00,24.0,1,27.0,68.0,108.0,16.000000,17.884474
3,2024-02-29 03:00:00,26.0,1,27.0,68.0,98.0,15.400000,17.864035
4,2024-02-29 04:00:00,19.0,1,27.0,70.0,100.0,17.799999,18.082321
...,...,...,...,...,...,...,...,...
139,2024-03-05 19:00:00,5.0,1,29.0,69.0,116.0,23.299999,18.135102
140,2024-03-05 20:00:00,7.0,1,29.0,72.0,111.0,29.500000,18.023005
141,2024-03-05 21:00:00,1.0,1,29.0,73.0,108.0,28.500000,18.252090
142,2024-03-05 22:00:00,5.0,1,28.0,74.0,105.0,23.799999,18.303057


In [15]:
import pandas as pd
from pycaret.regression import *

# Read the training data
train_data = pd.read_csv('train.csv')

# Convert 'DATETIMEDATA' column to datetime format
train_data['DATETIMEDATA'] = pd.to_datetime(train_data['DATETIMEDATA'])

# Set up the PyCaret regression environment
regression_setup = setup(train_data, target='PM25', session_id=123, date_features=['DATETIMEDATA'])

# Compare regression models and select the best one
best_model = compare_models()

# Create a Gradient Boosting Regressor model
rf_model = create_model('rf')

# Fine-tune the Random Forest Regressor model
tuned_rf_model = tune_model(best_model)

# Ensemble the tuned Random Forest Regressor model
bagged_model = ensemble_model(tuned_rf_model, n_estimators=20)

# Generate datetime values for the next week
start_date = pd.to_datetime('2024-03-06 00:00:00', format='%Y-%m-%d %H:%M:%S')  # Start date for next week
end_date = start_date + pd.DateOffset(days=6)  # End date for next week
next_week_dates = pd.date_range(start=start_date, end=end_date, freq='H')

# Create a DataFrame with 'DATETIMEDATA' column for the next week
next_week_data = pd.DataFrame({'DATETIMEDATA': next_week_dates})

# Predict PM25 for the next week
#next_week_predictions = predict_model(bagged_model, data=next_week_data)
modelpredict = predict_model(bagged_model, data=next_week_data)

mpdf = pd.DataFrame(modelpredict, columns = ['DATETIMEDATA', 'prediction_label'])

print(mpdf)
#print(next_week_predictions)


,Description,Value
0,Session id,123
1,Target,PM25
2,Target type,Regression
3,Original data shape,"(1416, 7)"
4,Transformed data shape,"(1416, 9)"
5,Transformed train set shape,"(991, 9)"
6,Transformed test set shape,"(425, 9)"
7,Numeric features,5
8,Date features,1
9,Preprocess,True


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,3.2689,18.4847,4.2819,0.3568,0.2244,0.1979,0.0450
et,Extra Trees Regressor,3.2589,18.8994,4.3321,0.3408,0.2258,0.1954,0.0330
lightgbm,Light Gradient Boosting Machine,3.4173,20.2593,4.4864,0.2958,0.2353,0.2062,0.2240
gbr,Gradient Boosting Regressor,3.4863,20.3491,4.5024,0.2901,0.2387,0.2137,0.0170
ada,AdaBoost Regressor,3.9353,23.9622,4.8766,0.1685,0.2670,0.2560,0.0090
lr,Linear Regression,3.9268,24.9870,4.9900,0.1302,0.2687,0.2455,0.0130
ridge,Ridge Regression,3.9269,24.9864,4.9899,0.1302,0.2687,0.2455,0.0070
br,Bayesian Ridge,3.9312,25.0087,4.9917,0.1300,0.2688,0.2458,0.0060
lar,Least Angle Regression,3.9501,25.1676,5.0104,0.1218,0.2697,0.2466,0.0060
huber,Huber Regressor,3.9657,25.9580,5.0839,0.0980,0.2707,0.2420,0.0080


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,3.7743,22.4586,4.7390,0.2250,0.2598,0.2410
1,3.3407,23.4027,4.8376,0.2125,0.2362,0.1981
2,3.3224,16.9486,4.1169,0.3658,0.2289,0.2055
3,3.5222,20.4020,4.5169,0.3159,0.2307,0.2022
4,2.9003,14.2941,3.7808,0.5001,0.2037,0.1783
5,3.3115,18.5914,4.3118,0.4264,0.2168,0.1904
6,2.8409,13.3942,3.6598,0.4529,0.1924,0.1644
7,3.2414,19.5181,4.4179,0.4026,0.2221,0.1904
8,2.8858,15.0213,3.8757,0.2894,0.2000,0.1722


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,3.9257,23.3391,4.8311,0.1946,0.2696,0.2557
1,3.3274,21.9185,4.6817,0.2624,0.2291,0.1953
2,3.4521,18.0441,4.2478,0.3248,0.2343,0.2124
3,3.7163,21.5591,4.6432,0.2771,0.2406,0.2161
4,3.1617,16.5300,4.0657,0.4219,0.2244,0.1988
5,3.3965,19.1196,4.3726,0.4101,0.2205,0.1966
6,2.9932,14.1105,3.7564,0.4236,0.2004,0.1769
7,3.5031,22.5751,4.7513,0.3090,0.2383,0.2065
8,2.8746,14.7914,3.8460,0.3003,0.1991,0.1705


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,3.7923,21.7999,4.6690,0.2477,0.2596,0.2458
1,3.3709,22.7703,4.7718,0.2337,0.2329,0.1987
2,3.4040,17.0834,4.1332,0.3607,0.2276,0.2094
3,3.5795,20.5612,4.5344,0.3105,0.2333,0.2078
4,3.0559,15.1118,3.8874,0.4715,0.2131,0.1906
5,3.3805,19.3078,4.3941,0.4043,0.2233,0.1988
6,2.9818,13.8146,3.7168,0.4357,0.1969,0.1743
7,3.4145,21.1023,4.5937,0.3541,0.2316,0.2027
8,2.7874,14.4636,3.8031,0.3158,0.1966,0.1663


           DATETIMEDATA  prediction_label
0   2024-03-06 00:00:00         22.112269
1   2024-03-06 01:00:00         22.112269
2   2024-03-06 02:00:00         22.112269
3   2024-03-06 03:00:00         22.112269
4   2024-03-06 04:00:00         22.112269
..                  ...               ...
140 2024-03-11 20:00:00         22.313977
141 2024-03-11 21:00:00         22.313977
142 2024-03-11 22:00:00         22.313977
143 2024-03-11 23:00:00         22.313977
144 2024-03-12 00:00:00         22.542177

[145 rows x 2 columns]
